In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.tabular.all import *
from wwf.tab.export import *
from bnbsl import *

from classes import *
from utils import *

In [3]:
SEASON = '22'
set_seed(int(SEASON))

## Download HKJC odds

In [4]:
path_raw = Path('raw_data')
path_data = Path('data')

path_output = Path('output')

In [5]:
session = requests.Session()
r = session.get('http://bet.hkjc.com')
cookies = r.cookies

In [6]:
odds_url = 'https://bet.hkjc.com/football/getJSON.aspx?jsontype=odds_chl.aspx'
response = session.post(
    odds_url,
    headers={'referer':'http://bet.hkjc.com'},
    cookies=cookies
)

In [7]:
with open(path_data/'json'/f'odds_chl-{datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.txt', 'w') as f:
    f.write(response.text)

In [8]:
matches_json = json.loads(response.text)['matches']

In [9]:
# Example
# matches_json = json.loads(open(path_data/'json'/'odds_chl-2021-11-20-20-15-14.txt').read())[1]['matches']

In [10]:
matches = [Match(m) for m in matches_json if m['matchStatus'] == 'Defined']
odds = [m.export() for m in matches]

['20230104WEDFB5999', '2023-Jan-04', '02:00:00', 'Spanish Cup [SFC]', 'Espanyol', 'Celta Vigo', 'true', '9.5', '1.97', '1.74', 'false', '10.5', '2.48', '1.47', 'false', '12.5', '4.25', '1.17']
['20230104WEDFB5976', '2023-Jan-04', '03:45:00', 'Eng Premier [EPL]', 'Arsenal', 'Newcastle', 'true', '10.5', '1.95', '1.75', 'false', '11.5', '2.45', '1.48', 'false', '13.5', '4.15', '1.18']
['20230104WEDFB5973', '2023-Jan-04', '03:45:00', 'Eng Premier [EPL]', 'Everton', 'Brighton', 'true', '10.5', '2.05', '1.68', 'false', '11.5', '2.65', '1.42', 'false', '13.5', '4.55', '1.15']
['20230104WEDFB5975', '2023-Jan-04', '03:45:00', 'Eng Premier [EPL]', 'Leicester', 'Fulham', 'true', '10.5', '2.08', '1.66', 'false', '11.5', '2.68', '1.41', 'false', '12.5', '3.45', '1.25']
['20230104WEDFB5974', '2023-Jan-04', '04:00:00', 'Eng Premier [EPL]', 'Manchester Utd', 'Bournemouth', 'true', '10.5', '1.85', '1.85', 'false', '11.5', '2.33', '1.53', 'false', '14.5', '5.10', '1.12']
['20230104WEDFB5990', '2023-Jan-

In [11]:
cols_match = ['MatchDay', 'Date', 'Time', 'LeagueJC', 'HomeTeamJC', 'AwayTeamJC']
cols_odds0 = ['MAINLINE_0', 'CHL_LINE_0', 'CHL_H_0', 'CHL_L_0']
cols_odds1 = ['MAINLINE_1', 'CHL_LINE_1', 'CHL_H_1', 'CHL_L_1']
cols_odds2 = ['MAINLINE_2', 'CHL_LINE_2', 'CHL_H_2', 'CHL_L_2']
cols_odds  = ['MAINLINE', 'CHL_LINE', 'CHL_H', 'CHL_L']
cols_pred  = ['alpha_1', 'alpha_2', 'mu_1', 'mu_2', 'omega', 'corr']

cols = cols_match + cols_odds0 + cols_odds1 + cols_odds2

In [12]:
odds = pd.DataFrame(odds, columns=cols)
odds = odds.fillna(value=np.nan)

In [13]:
cols_odds_ = cols_odds0[1:]+cols_odds1[1:]+cols_odds2[1:]
odds[cols_odds_] = odds[cols_odds_].astype(float)

In [14]:
odds

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE_0,CHL_LINE_0,CHL_H_0,CHL_L_0,MAINLINE_1,CHL_LINE_1,CHL_H_1,CHL_L_1,MAINLINE_2,CHL_LINE_2,CHL_H_2,CHL_L_2
0,20230104WEDFB5999,2023-Jan-04,02:00:00,Spanish Cup [SFC],Espanyol,Celta Vigo,true,9.5,1.97,1.74,false,10.5,2.48,1.47,false,12.5,4.25,1.17
1,20230104WEDFB5976,2023-Jan-04,03:45:00,Eng Premier [EPL],Arsenal,Newcastle,true,10.5,1.95,1.75,false,11.5,2.45,1.48,false,13.5,4.15,1.18
2,20230104WEDFB5973,2023-Jan-04,03:45:00,Eng Premier [EPL],Everton,Brighton,true,10.5,2.05,1.68,false,11.5,2.65,1.42,false,13.5,4.55,1.15
3,20230104WEDFB5975,2023-Jan-04,03:45:00,Eng Premier [EPL],Leicester,Fulham,true,10.5,2.08,1.66,false,11.5,2.68,1.41,false,12.5,3.45,1.25
4,20230104WEDFB5974,2023-Jan-04,04:00:00,Eng Premier [EPL],Manchester Utd,Bournemouth,true,10.5,1.85,1.85,false,11.5,2.33,1.53,false,14.5,5.10,1.12
5,20230104WEDFB5990,2023-Jan-04,19:30:00,Italian Division 1 [ISA],Salernitana,AC Milan,true,9.5,2.02,1.70,false,10.5,2.60,1.43,false,12.5,4.75,1.14
6,20230104WEDFB5987,2023-Jan-04,19:30:00,Italian Division 1 [ISA],Sassuolo,Sampdoria,true,9.5,2.13,1.63,false,10.5,2.80,1.38,false,12.5,4.90,1.13
7,20230104WEDFB5985,2023-Jan-04,21:30:00,Italian Division 1 [ISA],AC Torino,Verona,true,9.5,2.05,1.68,false,10.5,2.68,1.41,false,12.5,4.90,1.13
8,20230104WEDFB5981,2023-Jan-04,21:30:00,Italian Division 1 [ISA],Spezia,Atalanta,true,9.5,2.05,1.68,false,10.5,2.60,1.43,false,12.5,4.75,1.14
9,20230104WEDFB5986,2023-Jan-04,23:30:00,Italian Division 1 [ISA],Lecce,Lazio,true,9.5,2.05,1.68,false,10.5,2.68,1.41,false,12.5,4.90,1.13


In [15]:
odds['MatchDay'] = odds['MatchDay'].str[8:]
odds['Date'] = pd.to_datetime(odds['Date'])
odds['Time'] = pd.to_datetime(odds['Time'], format='%H:%M:%S').dt.time

In [16]:
odds['DateTimeJC'] = pd.to_datetime(odds['Date'].dt.date.map(str) + '-' + odds['Time'].map(str))
odds['DateTimeJC'] = odds['DateTimeJC'].dt.tz_localize('Hongkong')
odds['DateTime'] = odds['DateTimeJC'].dt.tz_convert('GB')

In [17]:
map_league = pd.read_csv(path_data/'league.csv')
map_team = pd.read_csv(path_data/'team.csv')

In [18]:
div = ['E0', 'E1', 'E2', 'D1', 'D2', 'SP1', 'SP2', 'I1', 'I2', 'F1', 'F2', 'UCL', 'UEC', 'UEF']
map_league = map_league[map_league.Div.isin(div)].reset_index(drop=True)

In [19]:
# Map Div name
odds = odds.merge(map_league[['LeagueJC', 'Div']], 'inner', on='LeagueJC')

In [20]:
# Map Team name
odds = odds.merge(map_team[['TeamNameJC', 'TeamName']].rename(columns={'TeamName':'HomeTeam'}), 'inner', 
                  left_on='HomeTeamJC', right_on='TeamNameJC').drop(columns=['TeamNameJC'])
odds = odds.merge(map_team[['TeamNameJC', 'TeamName']].rename(columns={'TeamName':'AwayTeam'}), 'inner', 
                  left_on='AwayTeamJC', right_on='TeamNameJC').drop(columns=['TeamNameJC'])

## Download recent stats

In [21]:
# Download latest results in current season
!wget -q https://www.football-data.co.uk/mmz4281/{SEASON}{int(SEASON)+1}/data.zip -O raw_data/data.zip

# Unzip to folder
!unzip -q -o raw_data/data.zip -d raw_data/{SEASON}

In [22]:
usecols = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'HC', 'AC', 'FTHG', 'FTAG', 'HS', 'AS', 'HST', 'AST']
dtype = {'HC':'float', 'AC':'float'}
parse_dates = ['Date']

seasons = [SEASON]

dfs = []

for folder in sorted(path_raw.iterdir()):
    if folder.is_dir() and folder.name in seasons: 
        for file in sorted(folder.glob('*.csv')):
            try:
                df = pd.read_csv(file, usecols=usecols, dtype=dtype, parse_dates=parse_dates, dayfirst=True)
                df['Season'] = folder.name
                dfs.append(df)
            except:
                continue

In [23]:
df_season = pd.concat(dfs)
df_season = df_season.dropna()
df_season = df_season.sort_values(['Div', 'Date', 'HomeTeam']).reset_index(drop=True)

In [24]:
df_hist = pd.read_csv(path_data/'data.csv', dtype={'HC':'float', 'AC':'float'}, parse_dates=['Date'])
df_hist = df_hist.query(f'Season == {int(SEASON)-1}').reset_index(drop=True)

In [25]:
df_season = pd.concat([df_hist[df_season.columns], df_season])

In [26]:
# Make features on historical stats (Home and Away)
stats = ['FTHG', 'HS', 'HST', 'HC', 'FTAG', 'AS', 'AST', 'AC']
df_home, df_away = joinLastGamesStatsHomeAway(df_season, stats)

In [27]:
# Make features on historical stats (For and Against)
stats = [('FTHG', 'FTAG', 'FTG'), ('HS', 'AS', 'S'), ('HST', 'AST', 'ST'), ('HC', 'AC', 'C')]
df_for, df_against = joinLastGamesStatsForAgainst(df_season, stats)

In [28]:
df_home = df_home.sort_values(['HomeTeam', 'Date']).reset_index(drop=True)
df_away = df_away.sort_values(['AwayTeam', 'Date']).reset_index(drop=True)

df_home = df_home.groupby('HomeTeam')[df_home.columns[df_home.columns.str.contains('Avg')]].last().reset_index()
df_away = df_away.groupby('AwayTeam')[df_away.columns[df_away.columns.str.contains('Avg')]].last().reset_index()

In [29]:
odds = odds.merge(df_home, 'left', 'HomeTeam').merge(df_away, 'left', 'AwayTeam')

In [30]:
cols_home = df_for.columns[df_for.columns.str.contains('Avg')]
cols_home = dict(zip(cols_home, 'Home'+cols_home))
cols_home.update({'Team':'HomeTeam'})

cols_away = df_for.columns[df_for.columns.str.contains('Avg')]
cols_away = dict(zip(cols_home, 'Away'+cols_away))
cols_away.update({'Team':'AwayTeam'})

df_for = df_for.groupby('Team')[df_for.columns[df_for.columns.str.contains('Avg')]].last().reset_index()
odds = odds.merge(df_for.rename(columns=cols_home), 'left', 'HomeTeam').merge(df_for.rename(columns=cols_away), 'left', 'AwayTeam')

In [31]:
cols_home = df_against.columns[df_against.columns.str.contains('Avg')]
cols_home = dict(zip(cols_home, 'Home'+cols_home))
cols_home.update({'Team':'HomeTeam'})

cols_away = df_against.columns[df_against.columns.str.contains('Avg')]
cols_away = dict(zip(cols_home, 'Away'+cols_away))
cols_away.update({'Team':'AwayTeam'})

df_against = df_against.groupby('Team')[df_against.columns[df_against.columns.str.contains('Avg')]].last().reset_index()
odds = odds.merge(df_against.rename(columns=cols_home), 'left', 'HomeTeam').merge(df_against.rename(columns=cols_away), 'left', 'AwayTeam')

In [32]:
add_datepart(odds, 'DateTime', prefix='', drop=False);

In [33]:
display_df(odds.head(5).T)

,0,1,2,3,4
MatchDay,WEDFB5976,WEDFB5973,WEDFB5975,WEDFB5974,THUFB5977
Date,2023-01-04 00:00:00,2023-01-04 00:00:00,2023-01-04 00:00:00,2023-01-04 00:00:00,2023-01-05 00:00:00
Time,03:45:00,03:45:00,03:45:00,04:00:00,03:30:00
LeagueJC,Eng Premier [EPL],Eng Premier [EPL],Eng Premier [EPL],Eng Premier [EPL],Eng Premier [EPL]
HomeTeamJC,Arsenal,Everton,Leicester,Manchester Utd,Southampton
AwayTeamJC,Newcastle,Brighton,Fulham,Bournemouth,Nottingham Forest
MAINLINE_0,true,true,true,true,true
CHL_LINE_0,10.5,10.5,10.5,10.5,10.5
CHL_H_0,1.95,2.05,2.08,1.85,2.0
CHL_L_0,1.75,1.68,1.66,1.85,1.72


## Load model

In [34]:
learn_bnb = load_learner('models/learn_bnbsl.pkl')

In [35]:
to = load_pandas('models/to.pkl')

In [36]:
def predict(self, row):
    "Predict on a Pandas Series"
    dl = self.dls.test_dl(row.to_frame().T)
    dl.dataset.conts = dl.dataset.conts.astype(np.float32)
    inp,preds,_ = self.get_preds(dl=dl, with_input=True, with_decoded=False)
    b = tuplify(inp)
    full_dec = self.dls.decode(b)
    return full_dec,preds[0]

learn_bnb.predict = MethodType(predict, learn_bnb)

In [37]:
odds = odds.dropna(how='any')

In [38]:
to_tst = to.new(odds)
to_tst.process()
# to_tst.items.head()

In [39]:
tst_dl = learn_bnb.dls.valid.new(to_tst)
tst_dl.show(max_n=999)

,Div,HomeTeam,AwayTeam,HSLast5Avg,HCLast5Avg,ASLast5Avg,ACLast5Avg,HomeSForLast5Avg,AwaySForLast5Avg,HomeSAgainstLast5Avg,AwaySAgainstLast5Avg
0,E0,Arsenal,Newcastle,20.2,7.4,10.6,4.2,16.0,12.4,7.8,9.8
1,E0,Everton,Brighton,10.6,5.6,12.2,5.6,11.4,12.0,15.6,11.0
2,E0,Leicester,Fulham,11.2,4.2,9.4,4.2,10.8,15.2,15.6,11.4
3,E0,Man United,Bournemouth,16.2,5.0,8.4,3.2,13.6,9.0,9.4,18.2
4,E0,Southampton,Nott'm Forest,13.4,6.6,7.2,3.4,12.6,9.4,10.6,14.0
5,E0,Leeds,West Ham,14.2,6.8,13.2,8.8,13.4,12.8,16.4,12.2
6,E0,Aston Villa,Wolves,11.4,4.8,12.8,4.4,9.6,12.2,12.6,12.4
7,E0,Crystal Palace,Tottenham,11.8,5.2,12.0,6.0,9.6,16.8,12.0,10.6
8,E0,Chelsea,Man City,9.2,5.4,16.2,6.4,9.2,19.2,13.0,8.2
9,I1,Salernitana,Milan,12.6,4.2,14.4,4.6,10.0,15.8,14.4,12.2


In [40]:
pred, _ = learn_bnb.get_preds(dl=tst_dl)

In [41]:
alpha = F.softplus(pred[:, 0:2])
mu = F.softplus(pred[:, 2:4])
omega = pred[:, 4]

total_count = 1. / alpha
logits = torch.log(alpha * mu)

# print(alpha, mu, omega)

In [42]:
# Method 1:
corr = omega * torch.sqrt(mu.prod(-1)) * alpha.prod(-1) / ((1.+alpha)**(1./alpha+1.)).prod(-1) / torch.sqrt((1.+alpha*mu).prod(-1))

In [43]:
# Method 2:
# d = 1.-math.exp(-1)
# corr = omega * d**2 * torch.sqrt(mu.prod(-1)*(1.+alpha*mu).prod(-1)) * ((1.+d*alpha*mu)**(-1-1/alpha)).prod(-1)

In [44]:
odds[cols_pred] = torch.cat([alpha, mu, omega.unsqueeze(-1), corr.unsqueeze(-1)], dim=-1)

In [45]:
odds0 = odds[cols_match+cols_odds0+cols_pred].rename(columns=dict(zip(cols_odds0, cols_odds)))
odds1 = odds[cols_match+cols_odds1+cols_pred].rename(columns=dict(zip(cols_odds1, cols_odds)))
odds2 = odds[cols_match+cols_odds2+cols_pred].rename(columns=dict(zip(cols_odds2, cols_odds)))

In [46]:
odds = pd.concat([odds0, odds1, odds2]).dropna().reset_index(drop=True)
odds['MAINLINE'] = np.where(odds['MAINLINE']=='true', True, False)

In [47]:
odds.head(5)

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,alpha_1,alpha_2,mu_1,mu_2,omega,corr
0,WEDFB5976,2023-01-04,03:45:00,Eng Premier [EPL],Arsenal,Newcastle,True,10.5,1.95,1.75,0.056810,0.137470,6.948405,3.394473,-49.269474,-0.161015
1,WEDFB5973,2023-01-04,03:45:00,Eng Premier [EPL],Everton,Brighton,True,10.5,2.05,1.68,0.051964,0.084561,6.078017,4.835248,-51.906963,-0.114974
2,WEDFB5975,2023-01-04,03:45:00,Eng Premier [EPL],Leicester,Fulham,True,10.5,2.08,1.66,0.076635,0.077652,4.970696,4.933991,-47.140347,-0.126183
3,WEDFB5974,2023-01-04,04:00:00,Eng Premier [EPL],Manchester Utd,Bournemouth,True,10.5,1.85,1.85,0.067396,0.076173,6.072832,3.745478,-43.940098,-0.100856
4,THUFB5977,2023-01-05,03:30:00,Eng Premier [EPL],Southampton,Nottingham Forest,True,10.5,2.00,1.72,0.041400,0.082476,6.750976,4.657643,-54.088551,-0.099129


In [48]:
odds['corr'].hist(grid=False);

In [49]:
prob_hilo = []

for r in list(zip(odds['alpha_1'], odds['alpha_2'], odds['mu_1'], odds['mu_2'], odds['omega'], odds['CHL_LINE'])):
    total_count = 1. / torch.tensor(r[0:2], device='cpu') 
    logits = torch.log(torch.tensor(r[0:2], device='cpu') * torch.tensor(r[2:4], device='cpu'))
    omega = torch.tensor(r[4], device='cpu')

    bnb_corner = BivariateNegativeBinomialSL(total_count=total_count, omega=omega, logits=logits)
    value = torch.cartesian_prod(torch.arange(0., 15.), torch.arange(0., 15.))
    corner = bnb_corner.log_prob(value).exp()
    
    line = r[5]
    mask = value.sum(-1) < line
    prob_lo = corner[mask].sum()
    prob_hi = 1 - prob_lo
    
    prob_hilo.append([prob_hi.item(), prob_lo.item()])

/Users/twtang/miniconda3/envs/fastai2/lib/python3.8/site-packages/torch/functional.py:1069: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1634272482218/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.cartesian_prod(tensors)  # type: ignore[attr-defined]


In [50]:
odds[['prob_hi', 'prob_lo']] = prob_hilo

In [51]:
odds['kelly_hi'] = (odds['prob_hi'] * odds['CHL_H'] - 1) / (odds['CHL_H'] - 1)
odds['kelly_lo'] = (odds['prob_lo'] * odds['CHL_L'] - 1) / (odds['CHL_L'] - 1)

In [52]:
odds['kelly'] = np.where(
    np.maximum(odds['kelly_hi'], odds['kelly_lo']) > 0, 
    np.where(odds['kelly_hi'] > odds['kelly_lo'], odds['kelly_hi'], odds['kelly_lo']), 
    np.nan
)

In [53]:
odds['bet'] = np.where(
    np.maximum(odds['kelly_hi'], odds['kelly_lo']) > 0, 
    np.where(odds['kelly_hi'] > odds['kelly_lo'], 'High', 'Low'), 
    None
)

In [54]:
odds = odds.sort_values('kelly', ascending=False).reset_index(drop=True)

In [55]:
odds['selected'] = np.where(
    odds['MAINLINE']==True, np.where(
        odds['kelly']>0.3, '$$$', np.where(
            odds['kelly']>0.2, '$$', np.where(
                odds['kelly']>0.1, '$', None))), 
    None
)

In [56]:
odds = odds.drop(columns=cols_pred+['kelly_hi', 'kelly_lo'])

In [57]:
odds[odds.bet.notna() & odds.selected.notna()]

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,prob_hi,prob_lo,kelly,bet,selected
0,WEDFB5987,2023-01-04,19:30:00,Italian Division 1 [ISA],Sassuolo,Sampdoria,True,9.5,2.13,1.63,0.647029,0.352971,0.334666,High,$$$
2,WEDFB5989,2023-01-04,23:30:00,Italian Division 1 [ISA],Roma,Bologna,True,9.5,1.91,1.79,0.656407,0.343593,0.278833,High,$$
6,WEDFB5981,2023-01-04,21:30:00,Italian Division 1 [ISA],Spezia,Atalanta,True,9.5,2.05,1.68,0.343794,0.656206,0.150627,Low,$
7,WEDFB5985,2023-01-04,21:30:00,Italian Division 1 [ISA],AC Torino,Verona,True,9.5,2.05,1.68,0.563793,0.436207,0.148357,High,$
9,THUFB5977,2023-01-05,03:30:00,Eng Premier [EPL],Southampton,Nottingham Forest,True,10.5,2.00,1.72,0.573142,0.426858,0.146285,High,$
10,WEDFB5974,2023-01-04,04:00:00,Eng Premier [EPL],Manchester Utd,Bournemouth,True,10.5,1.85,1.85,0.395397,0.604603,0.139430,Low,$
15,WEDFB5986,2023-01-04,23:30:00,Italian Division 1 [ISA],Lecce,Lazio,True,9.5,2.05,1.68,0.545881,0.454119,0.113386,High,$


In [58]:
display_df(odds[odds.MatchDay.isin(odds[odds.bet.notna() & odds.selected.notna()].MatchDay) & odds.bet.notna()])

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,prob_hi,prob_lo,kelly,bet,selected
0,WEDFB5987,2023-01-04,19:30:00,Italian Division 1 [ISA],Sassuolo,Sampdoria,True,9.5,2.13,1.63,0.647029,0.352971,0.334666,High,$$$
1,WEDFB5987,2023-01-04,19:30:00,Italian Division 1 [ISA],Sassuolo,Sampdoria,False,10.5,2.80,1.38,0.537016,0.462984,0.279802,High,None
2,WEDFB5989,2023-01-04,23:30:00,Italian Division 1 [ISA],Roma,Bologna,True,9.5,1.91,1.79,0.656407,0.343593,0.278833,High,$$
3,WEDFB5989,2023-01-04,23:30:00,Italian Division 1 [ISA],Roma,Bologna,False,10.5,2.45,1.48,0.547197,0.452803,0.234919,High,None
4,WEDFB5981,2023-01-04,21:30:00,Italian Division 1 [ISA],Spezia,Atalanta,False,10.5,2.60,1.43,0.244647,0.755353,0.186406,Low,None
5,WEDFB5987,2023-01-04,19:30:00,Italian Division 1 [ISA],Sassuolo,Sampdoria,False,12.5,4.90,1.13,0.328084,0.671916,0.155798,High,None
6,WEDFB5981,2023-01-04,21:30:00,Italian Division 1 [ISA],Spezia,Atalanta,True,9.5,2.05,1.68,0.343794,0.656206,0.150627,Low,$
7,WEDFB5985,2023-01-04,21:30:00,Italian Division 1 [ISA],AC Torino,Verona,True,9.5,2.05,1.68,0.563793,0.436207,0.148357,High,$
8,WEDFB5974,2023-01-04,04:00:00,Eng Premier [EPL],Manchester Utd,Bournemouth,False,11.5,2.33,1.53,0.295092,0.704908,0.148130,Low,None
9,THUFB5977,2023-01-05,03:30:00,Eng Premier [EPL],Southampton,Nottingham Forest,True,10.5,2.00,1.72,0.573142,0.426858,0.146285,High,$


In [59]:
odds[odds.MAINLINE].to_csv(path_output/f'odds-{datetime.now().strftime("%Y-%m-%d")}.csv', float_format='%.2f', index=False)